# Single Inference

In [17]:
from models import gender_model
from models import indoor_outdoor_model
from video_info import get_video_info
import cv2

# Importing Models Weights
face1 = "./weights/opencv_face_detector.pbtxt"
face2 = "./weights/opencv_face_detector_uint8.pb"
gen1 = "./weights/gender_deploy.prototxt"
gen2 = "./weights/gender_net.caffemodel"
weights = "./weights/idod.weights.h5"
path = '/content/sample_data3/data1000/v_PlayingCello_g08_c05.avi'

gender = gender_model(face2, face1, gen2, gen1)
idod = indoor_outdoor_model(weights)

#Get Video Info
duration_sec, fps, width, height, frame_count = get_video_info(path)

# Open the video file
cap = cv2.VideoCapture(path)

# Read until the video is completed
frame_count = 0
frame_skip_interval = fps
res = []
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        frame_count += 1
        if frame_count % frame_skip_interval == 0:
          #Dectect Human and gender
          res1 = gender.find_gender(frame)
          for i in res1:
            if i not in res:
              res.append(i)

          #Predict Environment
          if frame_count == frame_skip_interval:
            image = cv2.resize(frame, (112, 112))
            res2 = idod.find_environment(image)
    else:
        break

results = {'Title':path.split('/')[-1].split('_')[1],'Duration':duration_sec, 'FPS':fps, 'Width':width, 'Height':height,
                'Frames':frame_count,'Human':res, 'Environment':res2,'File Name':path.split('/')[-1]}

# Display Results
for key,res in results.items():
  print(key,"=", res)

1/1 [==============================] - 1s 847ms/step
Title = PlayingCello
Duration = 11.28
FPS = 25.0
Width = 320
Height = 240
Frames = 282
Human = ['Female']
Environment = Indoor
File Name = v_PlayingCello_g08_c05.avi


# Bulk Prediction

In [8]:
import os
from models import gender_model
from models import indoor_outdoor_model
from video_info import get_video_info
import cv2
import pandas as pd

# Importing Models Weights
face1 = "./weights/opencv_face_detector.pbtxt"
face2 = "./weights/opencv_face_detector_uint8.pb"
gen1 = "./weights/gender_deploy.prototxt"
gen2 = "./weights/gender_net.caffemodel"
weights = "./weights/idod.weights.h5"
folder_path = './data1000'

#load models
gender = gender_model(face2, face1, gen2, gen1)
idod = indoor_outdoor_model(weights)

#Create Dataframe
columns = ['Title','Duration', 'FPS', 'Width', 'Height', 'Frames','Human', 'Environment','File Name']
df = pd.DataFrame(columns=columns)

# List all files and directories in the specified folder
for file_name in os.listdir(folder_path):
  try:
    path =os.path.join(folder_path, file_name)

    #Get Video Info
    duration_sec, fps, width, height, frame_count = get_video_info(path)

    # Open the video file
    cap = cv2.VideoCapture(path)

    # Read until the video is completed
    frame_count = 0
    frame_skip_interval = fps
    res = []

    # Read video file
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame_count += 1

            #Dectect Human and gender
            if frame_count % frame_skip_interval == 0:
              res1 = gender.find_gender(frame)
              for i in res1:
                if i not in res:
                  res.append(i)

              #Predict Environment
              if frame_count == frame_skip_interval:
                image = cv2.resize(frame, (112, 112))
                res2 = idod.find_environment(image)
        else:
            break

    cap.release()
    # print(duration_sec, fps, width, height, frame_count,res, res2)
    # Example of appending new data
    new_data = {'Title':path.split('/')[-1].split('_')[1],'Duration':duration_sec, 'FPS':fps, 'Width':width, 'Height':height,
                'Frames':frame_count,'Human':res, 'Environment':res2,'File Name':path.split('/')[-1]}
    # Append new_data to the DataFrame
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
  except:
    print(path)

1/1 [==============================] - 0s 29ms/step
/content/sample_data3/data1000/v_HeadMassage_g07_c03.avi
1/1 [==============================] - 0s 29ms/step
/content/sample_data3/data1000/v_ApplyEyeMakeup_g17_c05.avi
1/1 [==============================] - 0s 44ms/step
/content/sample_data3/data1000/v_BrushingTeeth_g06_c05.avi
1/1 [==============================] - 0s 74ms/step
/content/sample_data3/data1000/v_Kayaking_g15_c01.avi
1/1 [==============================] - 0s 30ms/step
/content/sample_data3/data1000/v_ApplyLipstick_g15_c04.avi
1/1 [==============================] - 0s 30ms/step
/content/sample_data3/data1000/v_ApplyLipstick_g20_c02.avi
1/1 [==============================] - 0s 48ms/step
/content/sample_data3/data1000/v_BrushingTeeth_g09_c03.avi
1/1 [==============================] - 0s 49ms/step
/content/sample_data3/data1000/v_ApplyEyeMakeup_g21_c01.avi
1/1 [==============================] - 0s 28ms/step


In [9]:
df.head()

,Title,Duration,FPS,Width,Height,Frames,Human,Environment,File Name
0,MilitaryParade,5.520000,25.00000,320,240,138,[Male],Outdoor,v_MilitaryParade_g13_c01.avi
1,JumpingJack,3.203200,29.97003,320,240,96,[],Outdoor,v_JumpingJack_g12_c04.avi
2,ParallelBars,5.320000,25.00000,320,240,133,[],Outdoor,v_ParallelBars_g25_c01.avi
3,BaseballPitch,4.371033,29.97003,320,240,131,[],Outdoor,v_BaseballPitch_g13_c04.avi
4,ApplyEyeMakeup,4.080000,25.00000,320,240,102,[Female],Indoor,v_ApplyEyeMakeup_g24_c03.avi


In [10]:
len(df)

1032

In [11]:
df.to_csv("/content/sample_data/results.csv", index = False)